In [ ]:
# default_exp helpers

# Helpers

> Working with a DICOMweb API endpoint can involve a number of steps. This module provides helper functions. 

In [ ]:
#hide
from nbdev.showdoc import *

The Python Requests and HTTPX libraries do not natively support passing files as multipart/related in an HTTP request. Thus we need to create a helper function to encode a group of 'fields' (or files) in a body with portions separated with an appropriate boudary.  We can use the urllib3 library to provide much of the needed functionality.

In [ ]:
#export
from urllib3.filepost import encode_multipart_formdata, choose_boundary

def encode_multipart_related(fields, boundary=None):
    if boundary is None:
        boundary = choose_boundary()

    body, _ = encode_multipart_formdata(fields, boundary)
    content_type = str('multipart/related; boundary=%s' % boundary)

    return body, content_type

The DICOMweb standard provides a number of different REST endpoints that allow for efficient retrieval of DICOM metadata and image data. There is a firm structure for how you must query data, however. If you are interested in retrieving metadata about a Series (for instance), you must also pass the parent Study. To retrieve a list of Frames, you must pass the Study, Series and Instance that 'parent' the frames.

In [ ]:
#export
def construct_dicom_url_path(study_uid, series_uid = None, instance_uid = None, frames = None):        
    #if frames has a value, it's a single item or a list (which must be separated by commas)
    #  also, we must then also have a study, series and instance, 
    if frames:
        frame_string = ''
        if type(frames) is list:
            # iterate and separate with commas
            frame_string = ",".join(l)
        else:
            frame_string = frames
        return f'/studies/{study_uid}/series/{series_uid}/instances/{instance_uid}/frames/{frame_string}'

    #likewise, if frames=None and instance_uid exists, we must have study and series
    if instance_uid:
        return f'/studies/{study_uid}/series/{series_uid}/instances/{instance_uid}'

    #and so on with series
    if series_uid:
        return f'/studies/{study_uid}/series/{series_uid}'

    #must have at least the study
    return f'/studies/{study_uid}'

Likewise, when we are retrieving data, we must pass the correct header. We want to retrieve Frames as an octet-stream, Instances as application/dicom objects, and Series/Studies as multipart.

> TODO: This function currently assumes that the transfer-sytax is *. Update to support additional transfer syntaxes. Also note that * is NOT part of the DICOM standard, but is widely accepted by DICOMweb endpoints to return images in whatever encoding the image was initially provided.

In [ ]:
#export
def construct_retrieve_header(self, study_uid, series_uid = None, instance_uid = None, frames = None):

    #if frames has a value, it's a single item or a list (which must be separated by commas)
    #  also, we must them have a study, series and instance, 
    if frames:
        return {'Accept':'multipart/related; type="application/octet-stream"; transfer-syntax=*'}

    #likewise, if frames=None and instance_uid exists, we must have study and series
    if instance_uid:
        return {'Accept':'application/dicom'}

    #and so on with series
    if series_uid:
        return {'Accept':'multipart/related; type="application/dicom"; transfer-syntax=*'}

    #must have at least the study
    return {'Accept':'multipart/related; type="application/dicom"; transfer-syntax=*'}



When calling a DICOMweb endpoint, it's often useful to gather information about the request/response HTTP that was made. `get_http_response_dict` parses a Requests or HTTPX request and returns an ordered dict with information about the REST call.

In [ ]:
#export
def get_http_response_dict(self, response, is_binary = False, filename=None):
    d = OrderedDict()   # create a new ordered dictionary
    r = response

    # Add the information to the dictionary
    if filename:
        d['fname'] = str(filename)     
    d['method'] = r.request.method
    d['url'] = r.url
    d['path_url'] = r.request.path_url
    d['request_headers'] = str(r.request.headers) 
    if r.request.body:
        d['request_body_trimmed'] = r.request.body[0:150]   

    d['ok'] = r.ok
    d['status_code'] = r.status_code  
    d['reason'] = r.reason
    d['response_headers'] = r.headers
    if not is_binary:
        d['response_text'] = r.text  
        d['apparent_encoding'] = r.apparent_encoding
        d['encoding'] = r.encoding
    d['elapsed_time'] = r.elapsed

    return d  